In [4]:
import tweepy
import pandas as pd
import time
from typing import List, Dict, Optional

In [2]:
#Authentication with twitter API
API_KEY = 'qZA9z96WH1k5UAuX0QUjVDISI'
API_SECRET = 'pDcgyVxu9UDqbmz7KrmaBDazaX2db1sON3KGZyE2uaftsqmtW9'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAHF7xQEAAAAAnYIG%2Br2%2BGK658u8OB%2BdNXb232Sw%3DhcTeqVXsI7aAT2FxubvU0y43YGZbklrnGZ82e7dmBHRLDOWTaA'

client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

In [5]:
#Define a function to fetch tweets
def fetch_tweets(query, max_results=10):
	'''
	Fetch tweets using twitter API v2.

	Parameters:
		query (str): Search query (eg., keyword, hashtags).
		max_results (int): Number of tweets to retrive (max tweets per request)

	Returns:
		DataFrame: Tweets with metadata.
	'''
	tweets_data = []
	while True:
		try:
			response = client.search_recent_tweets(
				query=query,
				tweet_fields = ['created_at', 'public_metrics', 'author_id'],
				user_fields= ['public_metrics'],
				max_results = max_results,
				expansions='author_id'
			)

			if response.data:
				users = {u["id"]:u for u in response.includes["users"]}

				for tweet in response.data:
					author = users.get(tweet.author_id)
					metrics = tweet.public_metrics
					tweets_data.append({
						"date":tweet.created_at,
						"likeCount":metrics["like_count"],
						"replyCount":metrics["reply_count"],
						"retweetCount":metrics["retweet_count"],
						"viewCount":metrics.get("impression_count", None), #viewCount is experimental
						"followersCount": author.public_metrics["followers_count"],
						"tweets": tweet.text
					})
	
			return pd.DataFrame(tweets_data)
		except tweepy.TooManyRequests:
			print("Rate limit reached. waiting for 15 minutes...")
			time.sleep(15*60) # wait 15 minutes

query = "fashion -is:retweet"
tweets_df = fetch_tweets(query, max_results=10)

print(tweets_df.head())
tweets_df.to_csv("tweets_data.csv", index=False)

                       date  likeCount  replyCount  retweetCount  viewCount  \
0 2024-12-05 19:57:18+00:00          0           0             0          0   
1 2024-12-05 19:57:11+00:00          0           0             0          0   
2 2024-12-05 19:57:01+00:00          0           0             0          0   
3 2024-12-05 19:57:01+00:00          0           0             0          1   
4 2024-12-05 19:56:50+00:00          0           0             0          2   

   followersCount                                             tweets  
0            2707  @MaggyeHime I don't like the majority of what ...  
1              62  @obidoess @marysolsmojito @rianphin I think th...  
2            8262   @h0neyndmilk i owe Her my interest in fashion fr  
3            1685  KNC? out jt hanging with the real fashion girl...  
4              50  @_YazmeenSamiyya I think a decent percentage o...  
